In [10]:
import collections

import numpy as np
import tensorflow as tf
import tensorflow_federated as tff

import nest_asyncio
nest_asyncio.apply()

In [3]:
federated_float_on_clients = tff.FederatedType(tf.float32, tff.CLIENTS)

In [4]:
str(federated_float_on_clients.member)

'float32'

In [5]:
str(federated_float_on_clients.placement)

'CLIENTS'

In [6]:
str(federated_float_on_clients)

'{float32}@CLIENTS'

In [7]:
federated_float_on_clients.all_equal

False

In [8]:
simple_regression_model_type = (
    tff.NamedTupleType([('a', tf.float32), ('b', tf.float32)]))

str(simple_regression_model_type)

'<a=float32,b=float32>'

In [9]:
str(tff.FederatedType(
    simple_regression_model_type, tff.CLIENTS, all_equal=True))

'<a=float32,b=float32>@CLIENTS'

In [11]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def get_average_temperature(sensor_readings):
  return tff.federated_mean(sensor_readings)

In [12]:
str(get_average_temperature.type_signature)

'({float32}@CLIENTS -> float32@SERVER)'

In [13]:
get_average_temperature([68.5, 70.3, 69.8])

69.53334

In [14]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def get_average_temperature(sensor_readings):

  print ('Getting traced, the argument is "{}".'.format(
      type(sensor_readings).__name__))

  return tff.federated_mean(sensor_readings)

Getting traced, the argument is "ValueImpl".


In [16]:
@tff.tf_computation(tf.float32)
def add_half(x):
  return tf.add(x, 0.5)

print(add_half(5))

5.5


In [17]:
@tff.federated_computation(tff.FederatedType(tf.float32, tff.CLIENTS))
def add_half_on_clients(x):
  return tff.federated_map(add_half, x)

In [18]:
str(add_half_on_clients.type_signature)

'({float32}@CLIENTS -> {float32}@CLIENTS)'

In [19]:
add_half_on_clients([1.0, 3.0, 2.0])

[<tf.Tensor: shape=(), dtype=float32, numpy=1.5>,
 <tf.Tensor: shape=(), dtype=float32, numpy=3.5>,
 <tf.Tensor: shape=(), dtype=float32, numpy=2.5>]

In [20]:
try:

  # Eager mode
  constant_10 = tf.constant(10.)

  @tff.tf_computation(tf.float32)
  def add_ten(x):
    return x + constant_10

except Exception as err:
  print (err)

Attempting to capture an EagerTensor without building a function.


In [21]:
def get_constant_10():
  return tf.constant(10.)

@tff.tf_computation(tf.float32)
def add_ten(x):
  return x + get_constant_10()

add_ten(5.0)

15.0

In [22]:
float32_sequence = tff.SequenceType(tf.float32)

str(float32_sequence)

'float32*'

In [23]:
@tff.tf_computation(tff.SequenceType(tf.float32))
def get_local_temperature_average(local_temperatures):
  sum_and_count = (
      local_temperatures.reduce((0.0, 0), lambda x, y: (x[0] + y, x[1] + 1)))
  return sum_and_count[0] / tf.cast(sum_and_count[1], tf.float32)

In [24]:
str(get_local_temperature_average.type_signature)

'(float32* -> float32)'

In [25]:
@tff.tf_computation(tff.SequenceType(tf.int32))
def foo(x):
  return x.reduce(np.int32(0), lambda x, y: x + y)

foo([1, 2, 3])

6

In [26]:
get_local_temperature_average([68.5, 70.3, 69.8])

69.53333

In [27]:
@tff.tf_computation(tff.SequenceType(collections.OrderedDict([('A', tf.int32), ('B', tf.int32)])))
def foo(ds):
  print('element_structure = {}'.format(ds.element_spec))
  return ds.reduce(np.int32(0), lambda total, x: total + x['A'] * x['B'])

element_structure = OrderedDict([('A', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('B', TensorSpec(shape=(), dtype=tf.int32, name=None))])


In [28]:
str(foo.type_signature)

'(<A=int32,B=int32>* -> int32)'

In [29]:
foo([{'A': 2, 'B': 3}, {'A': 4, 'B': 5}])

26

In [30]:
@tff.federated_computation(
    tff.FederatedType(tff.SequenceType(tf.float32), tff.CLIENTS))
def get_global_temperature_average(sensor_readings):
  return tff.federated_mean(
      tff.federated_map(get_local_temperature_average, sensor_readings))

In [31]:
str(get_global_temperature_average.type_signature)

'({float32*}@CLIENTS -> float32@SERVER)'

In [32]:
get_global_temperature_average([[68.0, 70.0], [71.0], [68.0, 72.0, 70.0]])

70.0